In [1]:
!git clone https://github.com/harveyslash/Facial-Similarity-with-Siamese-Networks-in-Pytorch.git
!mkdir data
!mv Facial-Similarity-with-Siamese-Networks-in-Pytorch/data/faces/testing/* data/
!mv Facial-Similarity-with-Siamese-Networks-in-Pytorch/data/faces/training/* data/

fatal: destination path 'Facial-Similarity-with-Siamese-Networks-in-Pytorch' already exists and is not an empty directory.
mkdir: cannot create directory ‘data’: File exists
mv: cannot stat 'Facial-Similarity-with-Siamese-Networks-in-Pytorch/data/faces/testing/*': No such file or directory
mv: cannot stat 'Facial-Similarity-with-Siamese-Networks-in-Pytorch/data/faces/training/*': No such file or directory


In [2]:
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
import glob

from aijack.attack import MI_FACE
from aijack.defense import GeneralMomentAccountant, DPSGDManager
from aijack.utils import NumpyDataset
from sklearn.metrics import roc_auc_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from aijack.attack.membership import ShadowMembershipInferenceAttack
from aijack.utils.utils import TorchClassifier, NumpyDataset

BASE = "data/"

In [4]:

#load mnist dataset
from tensorflow.keras.datasets import mnist
#import shuffle from sklearn
from sklearn.utils import shuffle
#one hot encode from sklearn
from sklearn.preprocessing import OneHotEncoder

np.random.seed(42)
torch.manual_seed(42)


(X_train, y_train), (X_test, y_test) = mnist.load_data()

#shuffle the dataset
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

# downsample X_train and y_train to 1000 samples
X_train = X_train[:1000]
y_train = y_train[:1000]

X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)
# reshaping data
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

#normalizing data
X_train /= 255
X_test /= 255

#splitting train dataset into train and shadow dataset
X_train, X_shadow, y_train, y_shadow = train_test_split(
    X_train, y_train, test_size=1 / 2, random_state=42
)


# downsample X_test and y_test to 100 samples
X_test = X_test[:100]
y_test = y_test[:100]

#converting labels to int64
y_train = y_train.astype(np.int64)
y_test = y_test.astype(np.int64)
y_shadow = y_shadow.astype(np.int64)


# We simulate the situation where the distribution of training dataset is different from the test/shadow datasets.
X_test = 0.5 * X_test + 0.5 * np.random.normal(size=(X_test.shape))
X_test=X_test.astype(np.float32)


In [5]:
trainset = NumpyDataset(X_train, y_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=4, shuffle=True, num_workers=2
)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lin1 = nn.Linear(28 * 28, 10)

    def forward(self, x):
        out = self.lin1(x)
        return out
    
# Train the victim
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

criterion = nn.CrossEntropyLoss()
net = Net().to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [7]:
lot_size = 40
batch_size = 1
iterations = 10
sigma = 0.5
l2_norm_clip = 1
delta = 1e-5

accountant = GeneralMomentAccountant(
    noise_type="Gaussian",
    search="ternary",
    precision=0.001,
    order_max=1,
    order_min=72,
    max_iterations=1000,
    bound_type="rdp_upperbound_closedformula",
    backend="python",
)

privacy_manager = DPSGDManager(
    accountant,
    optim.SGD,
    l2_norm_clip=l2_norm_clip,
    dataset=trainset,
    lot_size=lot_size,
    batch_size=batch_size,
    iterations=iterations,
)

accountant.reset_step_info()
accountant.add_step_info(
    {"sigma": sigma},
    lot_size / len(trainset),
    iterations * (len(trainset) / lot_size),
)
estimated_epsilon = accountant.get_epsilon(delta=delta)
print(f"estimated epsilon is {estimated_epsilon}")

accountant.reset_step_info()
dpoptimizer_cls, lot_loader, batch_loader = privacy_manager.privatize(
    noise_multiplier=sigma
)

net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = dpoptimizer_cls(net.parameters(), lr=0.05, momentum=0.9)

for epoch in range(iterations):  # loop over the dataset multiple times
    running_loss = 0
    data_size = 0
    preds = []
    labels = []

    for X_lot, y_lot in lot_loader(optimizer):
        for X_batch, y_batch in batch_loader(TensorDataset(X_lot, y_lot)):
            optimizer.zero_grad()
            pred = net(X_batch)
            loss = criterion(pred, y_batch.to(torch.int64))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            data_size += X_batch.shape[0]
            preds.append(pred)
            labels.append(y_batch)

    preds = torch.cat(preds)
    labels = torch.cat(labels)
    print(f"epoch {epoch}: loss is {running_loss/data_size}")

print(f"final epsilon is {accountant.get_epsilon(delta=delta)}")


estimated epsilon is 32.73221302406533
epoch 0: loss is 2.237553165785631
epoch 1: loss is 1.981709357368889
epoch 2: loss is 1.5667168422448403
epoch 3: loss is 1.263464651343347
epoch 4: loss is 1.1401791267085997
epoch 5: loss is 0.9982775874563448
epoch 6: loss is 0.8367263239896938
epoch 7: loss is 0.8118689777258317
epoch 8: loss is 0.7478223530409965
epoch 9: loss is 0.6460464630438666
final epsilon is 29.544385543562118


In [8]:
in_preds = []
in_label = []
with torch.no_grad():
    for data in trainloader:
        inputs, labels = data
        outputs = net(inputs)
        in_preds.append(outputs)
        in_label.append(labels)
    in_preds = torch.cat(in_preds)
    in_label = torch.cat(in_label)
print(
    "Test Accuracy is: ",
    accuracy_score(np.array(torch.argmax(in_preds, axis=1)), np.array(in_label)),
)

Test Accuracy is:  0.772


In [11]:
# You need to wrap the torch module with TorchClassifier
clf = TorchClassifier(
    net, criterion, optimizer, batch_size=64, epoch=100, device=device
)

In [12]:
# Train the attacker


def create_clf():
    _net = Net().to(device)
    _optimizer = optim.Adam(_net.parameters(), lr=0.001)
    return TorchClassifier(
        _net, criterion, _optimizer, batch_size=64, epoch=100, device=device
    )


shadow_models = [create_clf() for _ in range(2)]
attack_models = [SVC(probability=True) for _ in range(10)]

attacker = ShadowMembershipInferenceAttack(clf, shadow_models, attack_models)
attacker.fit(X_shadow, y_shadow)

In [13]:
# Get the attack result of membership inference
in_result = attacker.predict(clf.predict_proba(X_train), y_train)
out_result = attacker.predict(clf.predict_proba(X_test), y_test)

in_label = np.ones(in_result.shape[0])
out_label = np.zeros(out_result.shape[0])

att_acc=accuracy_score(
    np.concatenate([in_label, out_label]), np.concatenate([in_result, out_result])
)


att_pr=precision_score(
    np.concatenate([in_label, out_label]), np.concatenate([in_result, out_result])
)

att_r=recall_score(
    np.concatenate([in_label, out_label]), np.concatenate([in_result, out_result])
)

att_f1=f1_score(
    np.concatenate([in_label, out_label]), np.concatenate([in_result, out_result])
)

att_auc=roc_auc_score(
    np.concatenate([in_label, out_label]), np.concatenate([in_result, out_result])
)

print("Accuracy: ",att_acc)
print("Precision: ",att_pr)
print("Recall: ",att_r)
print("F1: ",att_f1)
print("AUC: ",att_auc)

Accuracy:  0.5233333333333333
Precision:  0.9246031746031746
Recall:  0.466
F1:  0.6196808510638299
AUC:  0.638
